# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**

**Fecha**: 04/04/4045

**Nombre del equipo**: Equipo 2.0

**Profesor**: Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Files") \
    .master("spark://b1a47194426a:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

In [ ]:
log_lines = spark \
                .readStream \
                .format("text") \
                .option("maxFilesPerTrigger", 1) \
                .load("/home/jovyan/notebooks/data/log_streaming/input/")


In [ ]:
from pyspark.sql.functions import split

log_df = log_lines.select(split(log_lines.value, " | ").alias("logs_array"))
log_df = log_df.withColumn("timestamp", log_df["logs_array"].getItem(0).cast("string"))
log_df = log_df.withColumn("level", log_df["logs_array"].getItem(1).cast("string"))
log_df = log_df.withColumn("message", log_df["logs_array"].getItem(2).cast("string"))
log_df = log_df.withColumn("server", log_df["logs_array"].getItem(3).cast("string"))

log_df.printSchema()

In [ ]:
log_df = log_df.filter(log_df['level'] == "ERROR")

In [ ]:
query_files = log_df \
                .writeStream \
                .format("parquet") \
                .trigger(processingTime='10 seconds') \
                .option("path", "/home/jovyan/notebooks/data/log_streaming/output/") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .start()
query_files.awaitTermination(60)

In [ ]:
speed_df = spark \
            .read \
            .parquet("/home/jovyan/notebooks/data/log_streaming/output/")

speed_df.show(4, False)

In [ ]:
sc.stop()